In [1]:
import numpy as np
from qiskit import QuantumCircuit, execute, Aer
from math import pi

In [2]:
# reference: https://qiskit.org/textbook/ch-algorithms/quantum-phase-estimation.html

def qft_dagger(qc, n):
    """n-qubit QFTdagger the first n qubits in circ"""
    # Don't forget the Swaps!
    for qubit in range(n//2):
        qc.swap(qubit, n-qubit-1)
    for j in range(n):
        for m in range(j):
            qc.cp(-pi/float(2**(j-m)), m, j)
        qc.h(j)

In [34]:
from functools import lru_cache

@lru_cache(maxsize=10)
def power(base, exponent):
    if exponent == 0:
        return 1
    return base*power(base, exponent-1)


def phase_estimation(qc, n, phi):
    qc.x(n-1)

    for i in range(n-1):
        qc.h(i)

    for i in range(n-1):
        qc.cp(2*pi*phi*power(2, i), i, n-1)

In [47]:
circuit = QuantumCircuit(5, 5)

phase_estimation(circuit, 5, 0.5*0.5*0.5)
qft_dagger(circuit, 4)

circuit.measure(list(range(5)), list(range(5)))
circuit.draw()

┌───┐                                    ┌───┐                         »
q_0: ┤ H ├─■────────────────────────────────X─┤ H ├─■──────────────■────────»
     ├───┤ │                                │ └───┘ │P(-π/2) ┌───┐ │        »
q_1: ┤ H ├─┼────────■─────────────────X─────┼───────■────────┤ H ├─┼────────»
     ├───┤ │        │                 │     │                └───┘ │P(-π/4) »
q_2: ┤ H ├─┼────────┼────────■────────X─────┼──────────────────────■────────»
     ├───┤ │        │        │              │                               »
q_3: ┤ H ├─┼────────┼────────┼──────■───────X───────────────────────────────»
     ├───┤ │P(π/4)  │P(π/2)  │P(π)  │P(2π) ┌─┐                              »
q_4: ┤ X ├─■────────■────────■──────■──────┤M├──────────────────────────────»
     └───┘                                 └╥┘                              »
c: 5/═══════════════════════════════════════╩═══════════════════════════════»
                                            4                               »
«                                        ┌─┐                        
«q_0: ───────────■───────────────────────┤M├────────────────────────
«                │                       └╥┘          ┌─┐           
«q_1: ─■─────────┼──────────────■─────────╫───────────┤M├───────────
«      │P(-π/2)  │        ┌───┐ │         ║           └╥┘     ┌─┐   
«q_2: ─■─────────┼────────┤ H ├─┼─────────╫──■─────────╫──────┤M├───
«                │P(-π/8) └───┘ │P(-π/4)  ║  │P(-π/2)  ║ ┌───┐└╥┘┌─┐
«q_3: ───────────■──────────────■─────────╫──■─────────╫─┤ H ├─╫─┤M├
«                                         ║            ║ └───┘ ║ └╥┘
«q_4: ────────────────────────────────────╫────────────╫───────╫──╫─
«                                         ║            ║       ║  ║ 
«c: 5/════════════════════════════════════╩════════════╩═══════╩══╩═
«                                         0            1       2  3

In [48]:
simulator = Aer.get_backend('qasm_simulator')
job = execute(circuit, simulator, shots=10000)
result = job.result()

In [49]:
result.get_counts(circuit)

{'10010': 10000}